In [64]:
import pandas as pd
from operator import itemgetter
from itertools import groupby
import collections

In [8]:
jsonDf = pd.read_json('332.json')

In [12]:
newDf = jsonDf.transpose()

In [13]:
newDf

,audioguide,audioguide_categories,booths,brands,categories,companies,config,exhibitors,fair_city,kiosks
status,200,200,200,200,200,200,200,200,200,200
data,[],[],"[{'id': 76759, 'deleted': 1, 'id_pavilion': 11...","[{'id': 472958, 'deleted': 0, 'name': 'Shimadz...","[{'id': 67344, 'deleted': 0, 'name': 'Důlní, h...","[{'id': 19242, 'state_code': 'DE', 'deleted': ...","{'products': {'enabled': False, 'label': '', '...","[{'id': 58937, 'deleted': 0, 'name': 'KPL Trad...",[],"[{'id': 22585, 'deleted': 0, 'id_pavilion': 11..."


In [20]:
exhDf = pd.DataFrame(newDf['exhibitors']['data'])

In [26]:
exhDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 847 entries, 0 to 846
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           847 non-null    int64 
 1   deleted      847 non-null    int64 
 2   name         847 non-null    object
 3   description  847 non-null    object
 4   phone        847 non-null    object
 5   email        847 non-null    object
 6   id_parent    847 non-null    int64 
 7   brands       847 non-null    object
 8   kiosks       847 non-null    object
 9   categories   847 non-null    object
 10  companies    847 non-null    object
 11  products     847 non-null    object
 12  url          847 non-null    object
 13  logo         847 non-null    object
dtypes: int64(3), object(11)
memory usage: 92.8+ KB


In [27]:
exhDf = exhDf.loc[:, ['name', 'description', 'phone', 'email', 'companies', 'url']]

In [33]:
exhDf.shape

(847, 6)

In [37]:
exhDf = exhDf.explode('companies')

In [38]:
exhDf = exhDf.fillna(0)

In [40]:
companiesDf = pd.DataFrame(newDf['companies']['data'])

In [43]:
merged_df = pd.merge(exhDf, companiesDf, how='left', left_on='companies', right_on='id')

In [45]:
merged_df = merged_df.loc[:, ['name_x', 'name_y', 'description', 'phone', 'email', 'state_code', 'url']]

In [47]:
merged_df = merged_df.fillna('')

In [49]:
merged_df = merged_df.rename(columns={'name_x': 'company_name', 'name_y': 'name'})

In [61]:
def get_email_info(email:str) -> str:
    return email.split('.')[-1]

In [62]:
merged_df['new_code'] = merged_df['email'].apply(get_email_info)

In [69]:
codes = merged_df[["new_code"]].groupby(merged_df["new_code"]).count()

In [72]:
def set_language(new_code:str) -> str:
    return "CZ" if new_code == 'cz' else ("SK" if new_code == 'sk' else "EN")

In [73]:
merged_df['language'] = merged_df['new_code'].apply(set_language)

In [75]:
merged_df.drop('new_code', axis=1, inplace=True)

In [78]:
merged_df = merged_df.drop_duplicates(subset=['email'], keep='first', inplace=False)

In [83]:
merged_df.head(5)

,company_name,name,description,phone,email,state_code,url,language
0,KPL Trading s.r.o.,BOECK Technology GmbH,ODJEHLOVÁNÍ. Odjehlovací nástroje a stroje: BO...,420-602761100,info@kpl-trading.cz,DE,www.brusivo-nastroje.cz,CZ
3,SHIMADZU Handels GmbH - organizační složka,Shimadzu Corporation,,420-284080221,cz@shimadzu.eu.com,JP,www.shimadzu.com,EN
4,ATAS elektromotory Náchod a.s.,,"Asynchronní a stejnosměrné motory do 1.5 kW, p...",420-491446111,marketing@atas.cz,,www.atas.cz,CZ
5,"VALPLAST CZ, spol. s r.o.",ALOK MASTERBATCHES PVT. LTD.,Dodavatel polymerních surovin a aditiv do plastů.,420-533433680,valplast@valplast.cz,IN,www.valplast.cz,CZ
23,ALTEG Bohemia s.r.o.,,,420-737239539,office@alteg.eu,,www.alteg.eu,EN


In [84]:
merged_df.to_csv('bvv.csv', encoding='utf-8', index=False)

In [87]:
codes.to_csv('codes.csv', encoding='utf-8', index=True)